In [5]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

print("API Key Loaded Successfully" if api_key else "API Key Not Found")

API Key Loaded Successfully


In [6]:
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You're a helpful assistant."},
        {
            "role": "user",
            "content": "Write a limerick about the Python programming language.",
        },
    ],
)

In [7]:
response = completion.choices[0].message.content
print(response)

In the world of coding, there's Python,  
Its syntax as clear as the dawn.  
With libraries vast,  
It’s unsurpassed,  
For programmers, it's love that’s lifelong.


In [8]:
# 

In [9]:
# 

In [26]:
from pydantic import BaseModel, Field

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

In [11]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday.",
        },
    ],
    response_format=CalendarEvent,
)

In [13]:
event = completion.choices[0].message.parsed
event.name

'Science Fair'

In [14]:
event.date

'Friday'

In [15]:
event.participants

['Alice', 'Bob']

In [16]:
# 

In [17]:
# 

In [18]:
import requests
import json

def get_weather(latitude, longitude):
    """This is a publically available API that returns the weather for a given location."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]

In [19]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for provided coordinates in celsius.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

In [20]:
system_prompt = "You are a helpful weather assistant."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What's the weather like in Paris today?"},
]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

In [22]:
completion.model_dump()

{'id': 'chatcmpl-BG3vlnWepmGDb94m2wjhnqc91j4aa',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'annotations': [],
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_uq8TblTVOcZae7zZPRjhA87w',
      'function': {'arguments': '{"latitude":48.8566,"longitude":2.3522}',
       'name': 'get_weather'},
      'type': 'function'}]}}],
 'created': 1743168057,
 'model': 'gpt-4o-2024-08-06',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_6dd05565ef',
 'usage': {'completion_tokens': 25,
  'prompt_tokens': 66,
  'total_tokens': 91,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [23]:
def call_function(name, args):
    if name == "get_weather":
        return get_weather(**args)

In [24]:
for tool_call in completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)

    result = call_function(name, args)
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )

In [27]:
class WeatherResponse(BaseModel):
    temperature: float = Field(
        description="The current temperature in celsius for the given location."
    )
    response: str = Field(
        description="A natural language response to the user's question."
    )

In [28]:
completion_2 = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    response_format=WeatherResponse,
)

In [31]:
completion_2.model_dump()

{'id': 'chatcmpl-BG3xXEl042J1i9xLaKyFlVM0AxEIm',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '{"temperature":14.6,"response":"The current temperature in Paris is 14.6°C. It\'s a relatively mild day, perfect for exploring the city!"}',
    'refusal': None,
    'role': 'assistant',
    'annotations': [],
    'audio': None,
    'function_call': None,
    'tool_calls': None,
    'parsed': {'temperature': 14.6,
     'response': "The current temperature in Paris is 14.6°C. It's a relatively mild day, perfect for exploring the city!"}}}],
 'created': 1743168167,
 'model': 'gpt-4o-2024-08-06',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_6dd05565ef',
 'usage': {'completion_tokens': 40,
  'prompt_tokens': 209,
  'total_tokens': 249,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_de

In [29]:
final_response = completion_2.choices[0].message.parsed
final_response.temperature

14.6

In [30]:
final_response.response

"The current temperature in Paris is 14.6°C. It's a relatively mild day, perfect for exploring the city!"